# Transformación de tablas

Llamos a los dataframes para poder utilizarlos

In [1]:
%store -r apple_price
%store -r microsoft_price
%store -r twitter_price
%store -r apple_earnings
%store -r microsoft_earnings
%store -r twitter_earnings

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
from apifunctions import *



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/danigomezlechonbarrachina/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


In [4]:
from apifunctions import quitar_dolar

## Tablas earnings

A estas tablas queremos quitarles el símbolo del dolar a los valores de EPS y queremos establecer los tipos de datos de las columnas

In [5]:
earnings_list=[apple_earnings,microsoft_earnings,twitter_earnings]
for i in earnings_list:
    quitar_dolar(i)

In [6]:
# las columnas datetime se quedan como objeto no se muy bien por que, luego volvemos a esta función 

In [7]:

for i in earnings_list:
    cambiar_tipo_datos(i)

## Tablas price

Primero quedemos poner todos los valores como float a excepción de la fecha

In [8]:
prices_list=[apple_price,twitter_price,microsoft_price]

In [9]:
for i in prices_list:
    
    cambiar_tipo_datos2(i)

Ahora voy a añadir una columna que se llame price change que calcule como ha cambiado el precio respecto al precio de cierre del día anterior

In [10]:
col_price_change(prices_list)

Vamos a redondear los valores a dos puntos decimales

In [11]:
for j in range(len(prices_list)):
    
    columnas=prices_list[j].columns

    for i in columnas:
        
        if i!="Date":
        
            prices_list[j][i]=prices_list[j][i].round(decimals=2)
    

    

In [12]:
prueba1=twitter_price[twitter_price["Price_change"].abs()>6].head(5)

In [18]:
for i in prueba1.Date:
    
    lista=[get_news_link("TWTR",i)]


In [19]:
lista

['https://fortune.com/2015/03/24/obama-chief-digital-officer/']

In [20]:
def fun(path):
    
    # returns the final component of a url
    f_url = o.path.basename(path)
      
    # convert the url into link
    return '<a href="{}">{}</a>'.format(path, f_url)

In [23]:
import os as o

In [26]:
fun("https://fortune.com/2015/03/24/obama-chief-digital-officer/")

'<a href="https://fortune.com/2015/03/24/obama-chief-digital-officer/"></a>'

In [27]:
prueba1

,Date,Open,High,Low,Close,Adj Close,Volume,Price_change
2,01/06/2015,36.27,39.45,36.04,38.76,38.76,33050800.0,6.54
9,01/15/2015,39.70,39.70,36.86,36.93,36.93,23042000.0,-7.33
21,02/03/2015,38.00,40.19,37.69,39.79,39.79,28395400.0,6.22
24,02/06/2015,46.12,48.50,45.80,48.01,48.01,102973000.0,16.36
55,03/24/2015,48.67,51.79,48.54,51.47,51.47,38181800.0,6.21
